# Standard imports

In [1]:
%load_ext autoreload
%autoreload 2


import scqubits as scq

from qiskit_metal.analyses.quantization.lumped_capacitive import load_q3d_capacitance_matrix
from qiskit_metal.analyses.quantization.lom_core_analysis import CompositeSystem, Cell, Subsystem, QuantumSystemRegistry
                        
from scipy.constants import speed_of_light as c_light

import matplotlib.pyplot as plt
%matplotlib inline

12:36AM 06s INFO [__init__]: TransmonBuilder with system_type TRANSMON registered to QuantumSystemRegistry
12:36AM 06s INFO [__init__]: FluxoniumBuilder with system_type FLUXONIUM registered to QuantumSystemRegistry
12:36AM 06s INFO [__init__]: TLResonatorBuilder with system_type TL_RESONATOR registered to QuantumSystemRegistry
12:36AM 06s INFO [__init__]: LumpedResonatorBuilder with system_type LUMPED_RESONATOR registered to QuantumSystemRegistry


# Quantum System Reegistry

In [2]:
QuantumSystemRegistry.registry()

{'TRANSMON': qiskit_metal.analyses.quantization.lom_core_analysis.TransmonBuilder,
 'FLUXONIUM': qiskit_metal.analyses.quantization.lom_core_analysis.FluxoniumBuilder,
 'TL_RESONATOR': qiskit_metal.analyses.quantization.lom_core_analysis.TLResonatorBuilder,
 'LUMPED_RESONATOR': qiskit_metal.analyses.quantization.lom_core_analysis.LumpedResonatorBuilder}

## Load the capacitance matrices for each qubit

In [3]:
# Q1
path1 = "./capacitance_matrices/Q1_cap_matrix.txt"
t1_mat, _, _, _ = load_q3d_capacitance_matrix(path1)

Imported capacitance matrix with UNITS: [fF] now converted to USER UNITS:[fF]                 from file:
	./capacitance_matrices/Q1_cap_matrix.txt


,a_connector_pad_Q1,b_connector_pad_Q1,c_connector_pad_Q1,cl_metal_Q1,ground_main_plane,pad_bot_Q1,pad_top_Q1
a_connector_pad_Q1,50.69,-0.34,-0.45,-0.01,-34.01,-13.50,-1.52
b_connector_pad_Q1,-0.34,50.75,-0.14,-0.01,-34.19,-1.50,-13.69
c_connector_pad_Q1,-0.45,-0.14,50.73,-0.05,-34.13,-13.51,-1.59
cl_metal_Q1,-0.01,-0.01,-0.05,16.01,-15.53,-0.12,-0.21
ground_main_plane,-34.01,-34.19,-34.13,-15.53,248.10,-33.24,-37.77
pad_bot_Q1,-13.50,-1.50,-13.51,-0.12,-33.24,90.53,-26.22
pad_top_Q1,-1.52,-13.69,-1.59,-0.21,-37.77,-26.22,83.74


In [4]:
# Q2
path2 = "./capacitance_matrices/Q2_cap_matrix.txt"
t2_mat, _, _, _ = load_q3d_capacitance_matrix(path2)

Imported capacitance matrix with UNITS: [fF] now converted to USER UNITS:[fF]                 from file:
	./capacitance_matrices/Q2_cap_matrix.txt


,a_connector_pad_Q2,b_connector_pad_Q2,c_connector_pad_Q2,cl_metal_Q2,d_connector_pad_Q2,ground_main_plane,pad_bot_Q2,pad_top_Q2
a_connector_pad_Q2,51.41,-0.38,-0.59,-0.01,-0.16,-34.37,-13.50,-1.50
b_connector_pad_Q2,-0.38,51.42,-0.16,-0.01,-0.58,-34.36,-1.50,-13.50
c_connector_pad_Q2,-0.59,-0.16,51.38,-0.05,-0.38,-34.31,-13.49,-1.50
cl_metal_Q2,-0.01,-0.01,-0.05,15.94,-0.15,-15.38,-0.11,-0.15
d_connector_pad_Q2,-0.16,-0.58,-0.38,-0.15,51.48,-34.29,-1.50,-13.50
ground_main_plane,-34.37,-34.36,-34.31,-15.38,-34.29,271.68,-30.08,-30.03
pad_bot_Q2,-13.50,-1.50,-13.49,-0.11,-1.50,-30.08,86.24,-23.77
pad_top_Q2,-1.50,-13.50,-1.50,-0.15,-13.50,-30.03,-23.77,86.26


In [5]:
# Q3
path3 = "./capacitance_matrices/Q3_cap_matrix.txt"
t3_mat, _, _, _ = load_q3d_capacitance_matrix(path3)

Imported capacitance matrix with UNITS: [fF] now converted to USER UNITS:[fF]                 from file:
	./capacitance_matrices/Q3_cap_matrix.txt


,a_connector_pad_Q3,b_connector_pad_Q3,c_connector_pad_Q3,cl_metal_Q3,d_connector_pad_Q3,ground_main_plane,pad_bot_Q3,pad_top_Q3
a_connector_pad_Q3,52.29,-0.42,-0.76,-0.01,-0.18,-34.97,-13.47,-1.52
b_connector_pad_Q3,-0.42,52.29,-0.18,-0.01,-0.76,-34.98,-1.52,-13.48
c_connector_pad_Q3,-0.76,-0.18,52.31,-0.06,-0.42,-34.94,-13.48,-1.52
cl_metal_Q3,-0.01,-0.01,-0.06,15.96,-0.16,-15.42,-0.10,-0.14
d_connector_pad_Q3,-0.18,-0.76,-0.42,-0.16,52.37,-34.91,-1.52,-13.48
ground_main_plane,-34.97,-34.98,-34.94,-15.42,-34.91,270.02,-27.93,-27.91
pad_bot_Q3,-13.47,-1.52,-13.48,-0.10,-1.52,-27.93,82.20,-22.03
pad_top_Q3,-1.52,-13.48,-1.52,-0.14,-13.48,-27.91,-22.03,82.21


In [6]:
# Q4
path4 = "./capacitance_matrices/Q4_cap_matrix.txt"
t4_mat, _, _, _ = load_q3d_capacitance_matrix(path4)

Imported capacitance matrix with UNITS: [fF] now converted to USER UNITS:[fF]                 from file:
	./capacitance_matrices/Q4_cap_matrix.txt


,a_connector_pad_Q4,b_connector_pad_Q4,c_connector_pad_Q4,cl_metal_Q4,ground_main_plane,pad_bot_Q4,pad_top_Q4
a_connector_pad_Q4,52.54,-0.44,-0.84,-0.01,-35.22,-13.46,-1.60
b_connector_pad_Q4,-0.44,52.44,-0.20,-0.01,-35.42,-1.56,-13.82
c_connector_pad_Q4,-0.84,-0.20,52.48,-0.06,-35.26,-13.47,-1.67
cl_metal_Q4,-0.01,-0.01,-0.06,16.01,-15.59,-0.10,-0.16
ground_main_plane,-35.22,-35.42,-35.26,-15.59,241.52,-27.95,-32.50
pad_bot_Q4,-13.46,-1.56,-13.47,-0.10,-27.95,80.67,-21.97
pad_top_Q4,-1.60,-13.82,-1.67,-0.16,-32.50,-21.97,74.13


## Create LOM Cells from Capacitance Matrix

In [19]:
# Cell 2: Transmon-2
opt2 = dict(
    node_rename = {'b_connector_pad_Q2': 'coupler23',
                   'c_connector_pad_Q2': 'readout2'}, 
    cap_mat = t2_mat,
    ind_dict = {('pad_top_Q2', 'pad_bot_Q2'): 12},  # junction inductance in nH
    jj_dict = {('pad_top_Q2', 'pad_bot_Q2'): 'j2'},
    cj_dict = {('pad_top_Q2', 'pad_bot_Q2'): 1}, # junction capacitance in fF

)
cell_2 = Cell(opt2)

# Cell 3: Transmon-3
opt3 = dict(
    node_rename = {'b_connector_pad_Q3': 'coupler23',
                   'c_connector_pad_Q3': 'readout3'}, 
    cap_mat = t3_mat,
    ind_dict = {('pad_top_Q3', 'pad_bot_Q3'): 12},  # junction inductance in nH
    jj_dict = {('pad_top_Q3', 'pad_bot_Q3'): 'j3'},
    cj_dict = {('pad_top_Q3', 'pad_bot_Q3'): 1}, # junction capacitance in fF

)
cell_3 = Cell(opt3)

## Create Qubit Subsystems

In [20]:
# subsystem 1: Transmon-2
transmon2 = Subsystem(name='transmon2', sys_type='TRANSMON', nodes=['j2'])

# subsystem 2: Transmon-3
transmon3 = Subsystem(name='transmon3', sys_type='TRANSMON', nodes=['j3'])

In [21]:
# Resonator Subsystems
q_opts = dict(
    Z0 = 50, # characteristic impedance in Ohm
    vp = 0.404314 * c_light # phase velocity 
)

# subsystem 3: Q2 readout resonator
ro2 = Subsystem(name='readout2', sys_type='TL_RESONATOR', nodes=['readout2'], q_opts=dict(f_res = 6.96, **q_opts))

# subsystem 4: Q3 readout resonator
ro3 = Subsystem(name='readout3', sys_type='TL_RESONATOR', nodes=['readout3'], q_opts=dict(f_res = 6.98, **q_opts))

# subsystem 15: Q2-Q3 coupling resonator
coup23 = Subsystem(name='coupler23', sys_type='TL_RESONATOR', nodes=['coupler23'], q_opts=dict(f_res = 7.36, **q_opts))

# Create Composite System

In [22]:
composite_sys = CompositeSystem(
    subsystems=[transmon2, transmon3, coup23, ro2, ro3], 
    cells=[cell_2, cell_3],
    grd_node='ground_main_plane',
    nodes_force_keep=['readout2', 'readout3', 'coupler23']
)

# Print the Circuit Graph
This method generates a graph consisting of different circuit nodes and then reduces this to form an equivalent capacitance matrix only consisting of the reduced set of components.

In [23]:
cg = composite_sys.circuitGraph()
print(cg)

node_jj_basis:
-------------

['j2', 'pad_bot_Q2', 'j3', 'pad_bot_Q3', 'coupler23', 'readout2', 'readout3', 'a_connector_pad_Q2', 'a_connector_pad_Q3', 'cl_metal_Q2', 'cl_metal_Q3', 'd_connector_pad_Q2', 'd_connector_pad_Q3']

nodes_keep:
-------------

['j2', 'j3', 'coupler23', 'readout2', 'readout3']


L_inv_k (reduced inverse inductance matrix):
-------------

                 j2        j3  coupler23  readout2  readout3
j2         0.083333  0.000000        0.0       0.0       0.0
j3         0.000000  0.083333        0.0       0.0       0.0
coupler23  0.000000  0.000000        0.0       0.0       0.0
readout2   0.000000  0.000000        0.0       0.0       0.0
readout3   0.000000  0.000000        0.0       0.0       0.0

C_k (reduced capacitance matrix):
-------------

                  j2         j3  coupler23   readout2   readout3
j2         54.614341   0.000000  -6.022198   6.023767   0.000000
j3          0.000000  51.756296  -6.018143   0.000000   6.015938
coupler23  -6.022198  -

# Get Hilbert space of the Composite System

In [24]:
hilbertspace = composite_sys.create_hilbertspace()
print(hilbertspace)

HilbertSpace:  subsystems
-------------------------

Transmon------------| [Transmon_5]
                    | EJ: 13621.792733898432
                    | EC: 361.764776983381
                    | ng: 0.001
                    | ncut: 22
                    | truncated_dim: 10
                    |
                    | dim: 45


Transmon------------| [Transmon_6]
                    | EJ: 13621.792733898432
                    | EC: 382.04618957580647
                    | ng: 0.001
                    | ncut: 22
                    | truncated_dim: 10
                    |
                    | dim: 45


Oscillator----------| [Oscillator_1]
                    | E_osc: 7360.0
                    | l_osc: None
                    | truncated_dim: 3
                    |
                    | dim: 3


Oscillator----------| [Oscillator_2]
                    | E_osc: 6960.0
                    | l_osc: None
                    | truncated_dim: 3
                    |
                  

In [25]:
# Convert the hilbert space into
# "Interaction Picture"
hilbertspace = composite_sys.add_interaction()
hilbertspace.hamiltonian()

Quantum object: dims = [[10, 10, 3, 3, 3], [10, 10, 3, 3, 3]], shape = (2700, 2700), type = oper, isherm = True
Qobj data =
[[-21069.94377723+0.00000000e+00j      0.        +9.72724112e-02j
       0.        +0.00000000e+00j ...      0.        +0.00000000e+00j
       0.        +0.00000000e+00j      0.        +0.00000000e+00j]
 [     0.        -9.72724112e-02j -14089.94377723+0.00000000e+00j
       0.        +1.37563963e-01j ...      0.        +0.00000000e+00j
       0.        +0.00000000e+00j      0.        +0.00000000e+00j]
 [     0.        +0.00000000e+00j      0.        -1.37563963e-01j
   -7109.94377723+0.00000000e+00j ...      0.        +0.00000000e+00j
       0.        +0.00000000e+00j      0.        +0.00000000e+00j]
 ...
 [     0.        +0.00000000e+00j      0.        +0.00000000e+00j
       0.        +0.00000000e+00j ... 104725.46523701+0.00000000e+00j
       0.        +4.78222943e+02j      0.        +0.00000000e+00j]
 [     0.        +0.00000000e+00j      0.        +0.0000000

In [26]:
hamiltonian_results = composite_sys.hamiltonian_results(hilbertspace, evals_count=30)

Finished eigensystem.     

system frequencies in GHz:
--------------------------
{'transmon2': 5.875345402531805, 'transmon3': 6.024975402956764, 'coupler23': 7.372511788398417, 'readout2': 6.96758399935126, 'readout3': 6.989434164823506}

Chi matrices in MHz
--------------------------
            transmon2   transmon3  coupler23      readout2      readout3
transmon2 -418.646319    1.502700  -3.407336 -5.453703e+00 -4.588201e-02
transmon3    1.502700 -447.200698  -4.470614 -3.150036e-02 -7.439598e+00
coupler23   -3.407336   -4.470614   4.225685 -3.458875e-02 -6.315097e-02
readout2    -5.453703   -0.031500  -0.034589  2.032900e+00 -1.216649e-07
readout3    -0.045882   -7.439598  -0.063151 -1.216649e-07  2.134685e+00


In [27]:
composite_sys.compute_gs()

           transmon2   transmon3   coupler23   readout2   readout3
transmon2   0.000000   18.985900   95.566620 -93.568702   1.506729
transmon3  18.985900    0.000000  100.649134   1.497971 -98.781480
coupler23  95.566620  100.649134    0.000000   7.540125   7.987558
readout2  -93.568702    1.497971    7.540125   0.000000   0.118880
readout3    1.506729  -98.781480    7.987558   0.118880   0.000000